In [1]:
# 1.2 CountVectorizer

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df = 1)

In [2]:
# 1.3 훈련용 문장
contents = ['상처받은 아이들은 너무 일찍 커버려',
            '내가 상처받은 거 아는 사람 불편해',
            '잘 사는 사람들은 좋은 사람 되기 쉬워',
            '아무 일도 아니야 괜찮아']

In [3]:
# 1.4 Twitter -> Okt
from konlpy.tag import Twitter
t = Twitter()

/home/aquila/venv/nltk/lib/python3.10/site-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [4]:
# 1.5 형태소 분석
contents_tokens = [t.morphs(row) for row in contents]
contents_tokens

[['상처', '받은', '아이', '들', '은', '너무', '일찍', '커버', '려'],
 ['내', '가', '상처', '받은', '거', '아는', '사람', '불편해'],
 ['잘', '사는', '사람', '들', '은', '좋은', '사람', '되기', '쉬워'],
 ['아무', '일도', '아니야', '괜찮아']]

In [5]:
# 1.6 다시 문장으로~
contents_for_vectorize = []

for content in contents_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
    contents_for_vectorize.append(sentence)

contents_for_vectorize

[' 상처 받은 아이 들 은 너무 일찍 커버 려',
 ' 내 가 상처 받은 거 아는 사람 불편해',
 ' 잘 사는 사람 들 은 좋은 사람 되기 쉬워',
 ' 아무 일도 아니야 괜찮아']

In [6]:
# 1.7 벡터라이즈~
X = vectorizer.fit_transform(contents_for_vectorize)
X

<4x17 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [7]:
num_samples, num_features = X.shape
num_samples, num_features

(4, 17)

In [8]:
# 1.8 확인
vectorizer.get_feature_names_out()

array(['괜찮아', '너무', '되기', '받은', '불편해', '사는', '사람', '상처', '쉬워', '아는',
       '아니야', '아무', '아이', '일도', '일찍', '좋은', '커버'], dtype=object)

In [9]:
# 1.9 의미가 이해가 되나요?
X.toarray().transpose()

array([[0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 2, 0],
       [1, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0]])

In [10]:
# 1.10 테스트용 문장
new_post = ['상처받기 싫어 괜찮아']
new_post_tokens = [t.morphs(row) for row in new_post]#t.morphs(new_post)
print(new_post_tokens)
new_post_for_vectorize = []

for content in new_post_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
    new_post_for_vectorize.append(sentence)

new_post_for_vectorize

[['상처', '받기', '싫어', '괜찮아']]


[' 상처 받기 싫어 괜찮아']

In [11]:
# 1.11 벡터로 표현
new_post_vec = vectorizer.transform(new_post_for_vectorize)
new_post_vec.toarray()

array([[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [12]:
# 1.12 유클리드 거리
import scipy as sp

def dist_raw(v1,v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())

In [13]:
# 1.13 결과
dist = [dist_raw(each, new_post_vec) for each in X]
dist

[2.449489742783178, 2.23606797749979, 3.1622776601683795, 2.0]

In [14]:
print("Best post is", dist.index(min(dist)), ", dist = ", min(dist))
print("test post is --> ", new_post)
print("best dist post is --> ", contents[dist.index(min(dist))])

Best post is 3 , dist =  2.0
test post is -->  ['상처받기 싫어 괜찮아']
best dist post is -->  아무 일도 아니야 괜찮아


In [15]:
# 1.14 유사도 - 벡터 사이의 거리

for i in range(len(contents)):
    print(X.getrow(i).toarray())
print('-----------------------')
print(new_post_vec.toarray())

[[0 1 0 1 0 0 0 1 0 0 0 0 1 0 1 0 1]]
[[0 0 0 1 1 0 1 1 0 1 0 0 0 0 0 0 0]]
[[0 0 1 0 0 1 2 0 1 0 0 0 0 0 0 1 0]]
[[1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0]]
-----------------------
[[1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]]


# 1.15 TF-IDF
한 문서에서 많이 등장한 단어에 가중치 (Term Freq.)
반면,
전체 문서에서 많이 나타나는 단어는 중요하지 않게 (Inverse Document Freq.)
-> TF-IDF

In [16]:
# 1.16 TfidVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=1, decode_error='ignore')

# 1.18 다시 적용

In [17]:
X = vectorizer.fit_transform(contents_for_vectorize)
num_samples, num_features = X.shape
num_samples, num_features

(4, 17)

# 1.19 확인

In [18]:
X.toarray().transpose()

array([[0.        , 0.        , 0.        , 0.5       ],
       [0.43671931, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.39264414, 0.        ],
       [0.34431452, 0.40104275, 0.        , 0.        ],
       [0.        , 0.50867187, 0.        , 0.        ],
       [0.        , 0.        , 0.39264414, 0.        ],
       [0.        , 0.40104275, 0.6191303 , 0.        ],
       [0.34431452, 0.40104275, 0.        , 0.        ],
       [0.        , 0.        , 0.39264414, 0.        ],
       [0.        , 0.50867187, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.5       ],
       [0.        , 0.        , 0.        , 0.5       ],
       [0.43671931, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.5       ],
       [0.43671931, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.39264414, 0.        ],
       [0.43671931, 0.        , 0.        , 0.        ]])

# 1.20 테스트 문장 적용

In [19]:
new_post_vec = vectorizer.transform(new_post_for_vectorize)
new_post_vec.toarray()

array([[0.78528828, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.6191303 , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]])

# 1.21 거리를 구하는 또 다른 방법 normalize

In [20]:
def dist_norm(v1, v2):
    v1_normalized = v1 / sp.linalg.norm(v1.toarray())
    v2_normalized = v2 / sp.linalg.norm(v2.toarray())

    delta = v1_normalized - v2_normalized

    return sp.linalg.norm(delta.toarray())

In [21]:
# 1.22 결과.. 거리값이 바뀌어 있다.

dist = [dist_norm(each, new_post_vec) for each in X]

print('Best post is ', dist.index(min(dist)), ", dist = ", min(dist))
print('test post is --> ', new_post)
print('best dist post is --> ', contents[dist.index(min(dist))])

Best post is  3 , dist =  1.1021396119773588
test post is -->  ['상처받기 싫어 괜찮아']
best dist post is -->  아무 일도 아니야 괜찮아


# 2. 실전

## 2.1 예전 함수를 가져오고

In [22]:
import urllib.request
import json
import datetime

In [47]:
def gen_search_url(api_node,search_text,start_num,disp_num):
    base = 'https://openapi.naver.com/v1/search'
    node = '/' + api_node + '.json'
    param_query = '?query=' + urllib.parse.quote(search_text)
    param_start = '&start=' + str(start_num)
    param_disp = '&display=' + str(disp_num)
    return base + node + param_query + param_disp + param_start

def get_result_onpage(url):
    request = urllib.request.Request(url)
    request.add_header('X-Naver-Client-Id', client_id)
    request.add_header('X-Naver-Client-Secret', client_secret)
    response = urllib.request.urlopen(request)
    print(f'{datetime.datetime.now()} Url Request Success')
    return json.loads(response.read().decode('utf-8'))

# 2.3 문장을 수집하자

In [25]:
client_id = 'JaT2Y5TwnkhA7HRktKli'
client_secret = 'IpL_lZ84LN'

url = gen_search_url('kin', '자율주행', 1, 10)
one_result = get_result_onpage(url)
one_result

2023-10-06 18:00:47.874642 Url Request Success


{'lastBuildDate': 'Fri, 06 Oct 2023 18:00:47 +0900',
 'total': 27289,
 'start': 1,
 'display': 10,
 'items': [{'title': '<b>자율주행</b> 자동차가 실생활',
   'link': 'https://kin.naver.com/qna/detail.naver?d1id=8&dirId=81104&docId=452926448&qb=7J6Q7Jyo7KO87ZaJ&enc=utf8&section=kin.qna&rank=1&search_sort=0&spq=1',
   'description': '<b>자율주행</b>자동차가 실생활에 쓰이고있나요? <b>자율주행</b>의 단계를 0단계부터 5단계로 구분한다고 할 때 지금 출시되는 차는 2단계 차량들이 이미 실생활에 보급되고 있습니다 테슬라나 현대자동차의 차량들이 2단계 <b>자율주행</b>차량들 입니다... '},
  {'title': '자동차 <b>자율주행</b>장치를 모방하여 선박의 자율... ',
   'link': 'https://kin.naver.com/qna/detail.naver?d1id=6&dirId=60205&docId=447300072&qb=7J6Q7Jyo7KO87ZaJ&enc=utf8&section=kin.qna&rank=2&search_sort=0&spq=1',
   'description': '자동차 <b>자율주행</b>장치를 모방하여 선박의 <b>자율 주행</b>장치를 특허출원을 했다면 특허등록이 될까요?  안녕하세요. 대한변리사회-네이버 지식iN 상담변리사 문춘오 입니다. 저희 답변이 도움이 되기를 바랍니다. 자동차 <b>자율주행</b>장치를... '},
  {'title': '<b>자율주행</b>자동차',
   'link': 'https://kin.naver.com/qna/detail.naver?d1id=8&dirId=81104&docId=454455536&qb=7J6Q7Jyo7KO87ZaJ&enc=utf8&

# 2.4 이렇게 접근~

In [26]:
one_result['items'][0]['description']

'<b>자율주행</b>자동차가 실생활에 쓰이고있나요? <b>자율주행</b>의 단계를 0단계부터 5단계로 구분한다고 할 때 지금 출시되는 차는 2단계 차량들이 이미 실생활에 보급되고 있습니다 테슬라나 현대자동차의 차량들이 2단계 <b>자율주행</b>차량들 입니다... '

# 2.5 태그 제거기 호출

In [27]:
def delete_tag(input_str):
    input_str = input_str.replace('<b>','')
    input_str = input_str.replace('</b>', "")
    return input_str

def get_description(pages):
    contents = []
    for sentences in pages['items']:
        contents.append(delete_tag(sentences['description']))


    return contents

# 2.6 컨텐츠를 list로

In [28]:
contents = get_description(one_result)
contents

['자율주행자동차가 실생활에 쓰이고있나요? 자율주행의 단계를 0단계부터 5단계로 구분한다고 할 때 지금 출시되는 차는 2단계 차량들이 이미 실생활에 보급되고 있습니다 테슬라나 현대자동차의 차량들이 2단계 자율주행차량들 입니다... ',
 '자동차 자율주행장치를 모방하여 선박의 자율 주행장치를 특허출원을 했다면 특허등록이 될까요?  안녕하세요. 대한변리사회-네이버 지식iN 상담변리사 문춘오 입니다. 저희 답변이 도움이 되기를 바랍니다. 자동차 자율주행장치를... ',
 '자율주행자동차 보급으로 사람들에게 생길 새로운 습관 자세히 많이 알려주세요!!  안녕하세요 도움되실 내용으로 답변드립니다. 1. 시간의 활용: 자율주행 자동차가 운전을 대신하게 되면, 운전 중이던 시간을 다른 활동에 활용할 수... ',
 '자율주행자동차 핵심내용 좀 알랴주세여  자율주행 자동차 상세 내용 아래 참고요 자율주행차 정의, 원리, 레벨, 핵심기술 4가지_총정리 - https://pt1000.tistory.com/m/92',
 '수행평가로 자율주행 자동차에 대해 조사하는중인데 주제중 지금까지 실현된 자율주행 자동차 라는 주제가 있어서 2개 이상으로 알려주세요!! 지금 자율주행 자동차가 adas시스템 운전자 보조시스템 즉 자율주행 래밸2 까지입니다.... ',
 '자율주행자동차에 관심이 많은 고3입니다. 제가 생기부로 희망분야를 쭉 기계공학으로 채웠고 인공지능, 코딩과 관련된 자율주행자동차 세특으로 거의 채웠는데 코딩으로 채운 생기부가 기계공학에 지원하는데 좋게 볼까요?... ',
 '저는 미래에 진로쪽을 커넥티드카나 자율주행 자동차쪽으로 가고싶었습니다.그러던중 정보통신을 생각하고있었습니다.그러던중 학원 선생님께서 굳이 그쪽으로 갈건데 다른과 가도될거같은데 왜 정보통신이야?이런거 물어봤는데 선뜻... ',
 '자율주행 프로그래밍언어에 파이썬, C, C++말고도 자바도 사용되나요?  현대차에서 자율주행 플랫폼 개발자 구인 광고입니다. 직무상세 Classic/Adaptive AUTOSAR 

# 2.7 형태소 분석

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt

t = Okt()
vectorizer = CountVectorizer(min_df = 1)

In [30]:
contents_tokens = [t.morphs(row) for row in contents]
contents_tokens

[['자율',
  '주행',
  '자동차',
  '가',
  '실생활',
  '에',
  '쓰이고있나요',
  '?',
  '자율',
  '주행',
  '의',
  '단계',
  '를',
  '0',
  '단계',
  '부터',
  '5',
  '단계',
  '로',
  '구분',
  '한',
  '다',
  '고',
  '할',
  '때',
  '지금',
  '출시',
  '되는',
  '차는',
  '2',
  '단계',
  '차량',
  '들',
  '이',
  '이미',
  '실생활',
  '에',
  '보급',
  '되고',
  '있습니다',
  '테슬라',
  '나',
  '현',
  '대',
  '자동차',
  '의',
  '차량',
  '들',
  '이',
  '2',
  '단계',
  '자율',
  '주행',
  '차량',
  '들',
  '입니다',
  '...'],
 ['자동차',
  '자율',
  '주행',
  '장치',
  '를',
  '모방',
  '하여',
  '선박',
  '의',
  '자율',
  '주행',
  '장치',
  '를',
  '특허',
  '출원',
  '을',
  '했다면',
  '특허',
  '등록',
  '이',
  '될까',
  '요',
  '?',
  '안녕하세요',
  '.',
  '대',
  '한',
  '변리',
  '사회',
  '-',
  '네이버',
  '지식',
  'iN',
  '상담',
  '변리사',
  '문',
  '춘오',
  '입니다',
  '.',
  '저희',
  '답변',
  '이',
  '도움',
  '이',
  '되',
  '기를',
  '바랍니다',
  '.',
  '자동차',
  '자율',
  '주행',
  '장치',
  '를',
  '...'],
 ['자율',
  '주행',
  '자동차',
  '보급',
  '으로',
  '사람',
  '들',
  '에게',
  '생길',
  '새로운',
  '습관',
  '자세히',
  '많이',
  '알려주세요',
  '!!',
  '

# 2.8 vectorize 후 다시 합치고

In [31]:
contents_for_vectorize = []

for content in contents_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word

    contents_for_vectorize.append(sentence)

contents_for_vectorize




[' 자율 주행 자동차 가 실생활 에 쓰이고있나요 ? 자율 주행 의 단계 를 0 단계 부터 5 단계 로 구분 한 다 고 할 때 지금 출시 되는 차는 2 단계 차량 들 이 이미 실생활 에 보급 되고 있습니다 테슬라 나 현 대 자동차 의 차량 들 이 2 단계 자율 주행 차량 들 입니다 ...',
 ' 자동차 자율 주행 장치 를 모방 하여 선박 의 자율 주행 장치 를 특허 출원 을 했다면 특허 등록 이 될까 요 ? 안녕하세요 . 대 한 변리 사회 - 네이버 지식 iN 상담 변리사 문 춘오 입니다 . 저희 답변 이 도움 이 되 기를 바랍니다 . 자동차 자율 주행 장치 를 ...',
 ' 자율 주행 자동차 보급 으로 사람 들 에게 생길 새로운 습관 자세히 많이 알려주세요 !! 안녕하세요 도움 되실 내용 으로 답변 드립니다 . 1 . 시간 의 활용 : 자율 주행 자동차 가 운전 을 대신 하게 되면 , 운전 중 이던 시간 을 다른 활동 에 활용 할 수 ...',
 ' 자율 주행 자동차 핵심 내용 좀 알랴 주세여 자율 주행 자동차 상세 내용 아래 참고 요 자율 주 행차 정의 , 원리 , 레벨 , 핵심 기술 4 가지 _ 총 정리 - https://pt1000.tistory.com/m/92',
 ' 수행평가 로 자율 주행 자동차 에 대해 조사 하는 중 인데 주제 중 지금 까지 실현 된 자율 주행 자동차 라는 주제가 있어서 2 개 이상 으로 알려주세요 !! 지금 자율 주행 자동차 가 adas 시스템 운전자 보조 시스템 즉 자율 주행 래 밸 2 까지 입니다 ....',
 ' 자율 주행 자동차 에 관심 이 많은 고 3 입니다 . 제 가 생기 부로 희망 분야 를 쭉 기계공학 으로 채웠고 인공 지능 , 코딩 과 관련 된 자율 주행 자동차 세특 으로 거의 채웠는데 코딩 으로 채운 생기 부가 기계공학 에 지원 하는데 좋게 볼까 요 ?...',
 ' 저 는 미래 에 진로 쪽 을 커넥티드카 나 자율 주행 자동차 쪽 으로 가고싶었습니다 . 그러던 중 정보 통신 을 생각 하고있었습니다 . 그러던 

# 2.9 결과

In [32]:
X = vectorizer.fit_transform(contents_for_vectorize)
X

<10x196 sparse matrix of type '<class 'numpy.int64'>'
	with 247 stored elements in Compressed Sparse Row format>

In [33]:
num_samples, num_features = X.shape
num_samples, num_features

(10, 196)

# 2.10 벡터라이즈 결과

In [34]:
vectorizer.get_feature_names_out()

array(['92', 'adaptive', 'adas', 'autosar', 'classic', 'com', 'https',
       'in', 'pt1000', 'sw', 'tistory', '가고싶었습니다', '가도', '가지', '갈건데',
       '같은', '같은데', '개발', '개발자', '거의', '경우', '관련', '관심', '광고', '구분', '구인',
       '구현', '굳이', '그러던', '극단', '기계공학', '기를', '기반', '기술', '까지', '께서',
       '날씨', '내용', '네이버', '놓칠', '다른', '다양한', '단계', '답변', '대신', '대해',
       '데이터', '도로', '도움', '되고', '되나요', '되는', '되면', '되실', '될거', '될까', '둬야',
       '드립니다', '등록', '따라', '라는', '레벨', '많은', '많이', '말고도', '모듈', '모방',
       '물어봤는데', '미래', '바랍니다', '발생', '변리', '변리사', '보급', '보조', '보통', '볼까',
       '부가', '부로', '부터', '분야', '뿐입니다', '사람', '사용', '사회', '상담', '상세',
       '새로운', '생각', '생기', '생길', '선뜻', '선박', '선생님', '성능', '세특', '센서',
       '소프트웨어', '수행평가', '습관', '시간', '시스템', '실생활', '실현', '쓰이고있나요', '아래',
       '아키텍처', '안녕하세요', '알랴', '알려주세요', '언어', '에게', '에도', '에서', '에서의',
       '운전', '운전자', '원리', '으로', '이구', '이던', '이런거', '이미', '이상', '이야', '이지',
       '인공', '인데', '인식', '입니다', '있습니다', '있어서', '있으며', '자동차', '자바', '자세히'

# 2.11 유사도 측정할 테스트 문장

In [35]:
new_post = ['현대차 자율주행 직무에 기계공학 전공이 많나요?']
new_post_tokens = [t.morphs(row) for row in new_post]

new_post_for_vectorize = []

for content in new_post_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
    
    new_post_for_vectorize.append(sentence)


new_post_for_vectorize

[' 현대차 자율 주행 직무 에 기계공학 전공 이 많나요 ?']

# 2.12 벡터라이즈

In [36]:
new_post_vec = vectorizer.transform(new_post_for_vectorize)
new_post_vec.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

# 2.13 유클리드 거리

In [37]:
import scipy as sp

def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())

In [38]:
dist = [dist_raw(each, new_post_vec) for each in X]
dist

[8.12403840463596,
 7.0710678118654755,
 6.782329983125268,
 6.082762530298219,
 7.681145747868608,
 6.708203932499369,
 6.244997998398398,
 6.855654600401044,
 7.0710678118654755,
 6.164414002968976]

# 2.14 결과~ 괜찮지 않나요?

In [39]:
print("Best post is", dist.index(min(dist)), ", dist = ", min(dist))
print("test post is --> ", new_post)
print("best dist post is --> ", contents[dist.index(min(dist))])

Best post is 3 , dist =  6.082762530298219
test post is -->  ['현대차 자율주행 직무에 기계공학 전공이 많나요?']
best dist post is -->  자율주행자동차 핵심내용 좀 알랴주세여  자율주행 자동차 상세 내용 아래 참고요 자율주행차 정의, 원리, 레벨, 핵심기술 4가지_총정리 - https://pt1000.tistory.com/m/92


# 2.15 거리를 구하는 또다른 방법 normalize

In [40]:
def dist_norm(v1, v2):
    v1_normalized = v1 / sp.linalg.norm(v1.toarray())
    v2_normalized = v2 / sp.linalg.norm(v2.toarray())

    delta = v1_normalized - v2_normalized

    return sp.linalg.norm(delta.toarray())

# 2.16 결과

In [41]:
dist = [dist_norm(each, new_post_vec) for each in X]

print("Best post is", dist.index(min(dist)), ", dist = ", min(dist))
print("test post is --> ", new_post)
print("best dist post is --> ", contents[dist.index(min(dist))])

Best post is 7 , dist =  0.9295570619955884
test post is -->  ['현대차 자율주행 직무에 기계공학 전공이 많나요?']
best dist post is -->  자율주행 프로그래밍언어에 파이썬, C, C++말고도 자바도 사용되나요?  현대차에서 자율주행 플랫폼 개발자 구인 광고입니다. 직무상세 Classic/Adaptive AUTOSAR 기반의 SW 플랫폼 아키텍처 및 자율주행 특화 SW 모듈 개발 자율주행... 


In [42]:
# 2.17 Tfidf
def tfidf(t, d, D):
    tf = float(d.count(t)) / sum(d.count(w) for w in set(d))
    idf = sp.log(float(len(D))) / (len([doc for doc in D if t in doc]))

    return tf * idf

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=1, decode_error='ignore')

In [44]:
X = vectorizer.fit_transform(contents_for_vectorize)
num_samples, num_features = X.shape
num_samples, num_features

(10, 196)

# 2.18 이제 눈으로 확인하는건 약간 무리

In [45]:
new_post_vec = vectorizer.transform(new_post_for_vectorize)
new_post_vec.toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.55271517, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [46]:
# 2.19 결과
dist = [dist_norm(each, new_post_vec) for each in X]

print("Best post is", dist.index(min(dist)), ", dist = ", min(dist))
print("test post is --> ", new_post)
print("best dist post is --> ", contents[dist.index(min(dist))])

Best post is 7 , dist =  1.1815535659900955
test post is -->  ['현대차 자율주행 직무에 기계공학 전공이 많나요?']
best dist post is -->  자율주행 프로그래밍언어에 파이썬, C, C++말고도 자바도 사용되나요?  현대차에서 자율주행 플랫폼 개발자 구인 광고입니다. 직무상세 Classic/Adaptive AUTOSAR 기반의 SW 플랫폼 아키텍처 및 자율주행 특화 SW 모듈 개발 자율주행... 
